In [5]:
import glob
from pandas import read_csv, concat, cut, crosstab
from numpy import round, clip, trace, tril, infty

# LISTAGEM DADOS

In [6]:
cursos = glob.glob(f'C:/Users/jullya leticia/Downloads/inep/censo_superior/**/MICRODADOS_CADASTRO_CURSOS_*.CSV')
ies = glob.glob(f'C:/Users/jullya leticia/Downloads/inep/censo_superior/**/MICRODADOS_CADASTRO_IES_*.CSV')

# SELEÇÃO COLUNAS DE INTERESSE

In [7]:
colunas_ies = ['NU_ANO_CENSO', 'CO_IES', 'NO_IES', 'SG_IES', 'NO_REGIAO_IES','SG_UF_IES', 'TP_CATEGORIA_ADMINISTRATIVA']
colunas_cursos =['NU_ANO_CENSO','CO_IES','SG_UF','QT_ING','QT_CONC','TP_ORGANIZACAO_ACADEMICA','TP_CATEGORIA_ADMINISTRATIVA',
                 'QT_ING_PROCESCPUBLICA','QT_CONC_PROCESCPUBLICA','QT_ING_PROCESCPRIVADA','QT_CONC_PROCESCPRIVADA',
                 'CO_CINE_AREA_GERAL','NO_CINE_AREA_GERAL']

# LEITURA DE DADOS

In [8]:
# Transformações iniciais nos dados de cursos
pipeline = lambda x: (
read_csv(x,encoding='ISO-8859-1',sep=';', usecols=colunas_cursos,low_memory=False)
.rename(columns = {'NU_ANO_CENSO':'ANO'})
.query('TP_CATEGORIA_ADMINISTRATIVA<=2 & CO_CINE_AREA_GERAL == 5 & ANO >=2011')
.groupby(['ANO', 'CO_IES','CO_CINE_AREA_GERAL','NO_CINE_AREA_GERAL'])
.agg(QT_ING = ('QT_ING', 'sum'), QT_CONC =('QT_CONC', 'sum'),
     QT_INGpb = ('QT_ING_PROCESCPUBLICA', 'sum'), QT_CONCpb =('QT_CONC_PROCESCPUBLICA', 'sum'),
     QT_INGpv = ('QT_ING_PROCESCPRIVADA', 'sum'), QT_CONCpv =('QT_CONC_PROCESCPRIVADA', 'sum'))
.reset_index()
)
df_cursos = concat((pipeline(f) for f in cursos))

In [9]:
# Transformações iniciais nos dados de IES
pipeline = lambda x: (
read_csv(x,encoding='ISO-8859-1',sep=';',usecols=colunas_ies,low_memory=False)
.rename(columns = {'NU_ANO_CENSO':'ANO_MIN'})
.query('TP_CATEGORIA_ADMINISTRATIVA<=2')
)
df_ies = concat((pipeline(f) for f in ies)).drop_duplicates(subset=['CO_IES'])
df_ies

,ANO_MIN,NO_REGIAO_IES,SG_UF_IES,TP_CATEGORIA_ADMINISTRATIVA,CO_IES,NO_IES,SG_IES
0,2009,Centro-Oeste,MT,1,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,UFMT
1,2009,Centro-Oeste,DF,1,2,UNIVERSIDADE DE BRASILIA,UNB
2,2009,Nordeste,SE,1,3,UNIVERSIDADE FEDERAL DE SERGIPE,UFS
3,2009,Norte,AM,1,4,UNIVERSIDADE FEDERAL DO AMAZONAS,UFAM
4,2009,Nordeste,PI,1,5,UNIVERSIDADE FEDERAL DO PIAUI,UFPI
...,...,...,...,...,...,...,...
2454,2020,Nordeste,PI,1,25277,UNIVERSIDADE FEDERAL DO DELTA DO PARNAIBA,UFDPAR
2455,2020,Centro-Oeste,GO,1,25282,UNIVERSIDADE FEDERAL DE JATAÍ,UFJ
2456,2020,Centro-Oeste,MT,1,25352,Universidade Federal de Rondonópolis,UFR
2563,2021,Sul,SC,2,24530,Faculdade da Polícia Militar de Santa Catarina,FAPOM


# INDICADORES DE EVASÃO

In [10]:
# Preparação dos dados
# Funções
xlag = lambda x: x.shift(5)
def dropout(x, y): return round(clip((1-(x/y))*100,0,100),2)
df = (
df_cursos.sort_values(['CO_IES', 'ANO'])
.assign(QT_ING5 = lambda x: x.groupby(['CO_IES','CO_CINE_AREA_GERAL','NO_CINE_AREA_GERAL']).QT_ING.transform(xlag),
EVASAO = lambda x: dropout(x.QT_CONC, x.QT_ING5),
QT_ING5pv = lambda x: x.groupby(['CO_IES','CO_CINE_AREA_GERAL','NO_CINE_AREA_GERAL']).QT_INGpv.transform(xlag),
EVASAOpv = lambda x: dropout(x.QT_CONCpv, x.QT_ING5pv),
QT_ING5pb = lambda x: x.groupby(['CO_IES','CO_CINE_AREA_GERAL','NO_CINE_AREA_GERAL']).QT_INGpb.transform(xlag),
EVASAOpb = lambda x: dropout(x.QT_CONCpb, x.QT_ING5pb))
.dropna(subset=['QT_ING5'])
.merge(df_ies, on='CO_IES', how='inner')
)
df.head(7)

,ANO,CO_IES,CO_CINE_AREA_GERAL,NO_CINE_AREA_GERAL,QT_ING,QT_CONC,QT_INGpb,QT_CONCpb,QT_INGpv,QT_CONCpv,...,QT_ING5pv,EVASAOpv,QT_ING5pb,EVASAOpb,ANO_MIN,NO_REGIAO_IES,SG_UF_IES,TP_CATEGORIA_ADMINISTRATIVA,NO_IES,SG_IES
0,2016,1,5,"Ciências naturais, matemática e estatística",296.0,62.0,215.0,27.0,81.0,23.0,...,95.0,75.79,107.0,74.77,2009,Centro-Oeste,MT,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,UFMT
1,2017,1,5,"Ciências naturais, matemática e estatística",334.0,76.0,227.0,48.0,107.0,25.0,...,74.0,66.22,118.0,59.32,2009,Centro-Oeste,MT,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,UFMT
2,2018,1,5,"Ciências naturais, matemática e estatística",289.0,65.0,208.0,38.0,81.0,23.0,...,69.0,66.67,126.0,69.84,2009,Centro-Oeste,MT,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,UFMT
3,2019,1,5,"Ciências naturais, matemática e estatística",337.0,72.0,240.0,44.0,97.0,28.0,...,68.0,58.82,175.0,74.86,2009,Centro-Oeste,MT,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,UFMT
4,2020,1,5,"Ciências naturais, matemática e estatística",220.0,36.0,134.0,21.0,86.0,15.0,...,85.0,82.35,209.0,89.95,2009,Centro-Oeste,MT,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,UFMT
5,2021,1,5,"Ciências naturais, matemática e estatística",245.0,26.0,172.0,19.0,73.0,7.0,...,81.0,91.36,215.0,91.16,2009,Centro-Oeste,MT,1,UNIVERSIDADE FEDERAL DE MATO GROSSO,UFMT
6,2016,2,5,"Ciências naturais, matemática e estatística",598.0,334.0,283.0,90.0,315.0,243.0,...,485.0,49.90,206.0,56.31,2009,Centro-Oeste,DF,1,UNIVERSIDADE DE BRASILIA,UNB


In [11]:
# TRANSFORMAR EM CSV
#df.to_csv('D:/OneDrive/PUC/Ciência de dados e Analytics/Engenharia de Dados/MVP/dados_ES.csv', index=False)

# EVASÃO POR ANO

In [12]:
# EVASÃO ESCOLA
rpv = df.groupby(['ANO']).\
agg(QT_CONC=('QT_CONC', 'sum'), QT_ING5=('QT_ING5', 'sum'),
    QT_CONCpv=('QT_CONCpv', 'sum'), QT_ING5pv=('QT_ING5pv', 'sum'),
    QT_CONCpb=('QT_CONCpb', 'sum'), QT_ING5pb=('QT_ING5pb', 'sum')).\
assign(EVASAO = lambda x: dropout(x.QT_CONC, x.QT_ING5),
       EVASAOpv = lambda x: dropout(x.QT_CONCpv, x.QT_ING5pv),
       EVASAOpb = lambda x: dropout(x.QT_CONCpb, x.QT_ING5pb))
rpv.filter(items = ['EVASAO', 'EVASAOpv','EVASAOpb'])

,EVASAO,EVASAOpv,EVASAOpb
ANO,,,
2016,58.79,7.87,28.07
2017,57.52,12.78,29.94
2018,59.91,58.58,64.18
2019,60.01,60.67,60.74
2020,68.57,65.59,71.03
2021,66.10,60.84,69.82


# POR TIPO DE ESCOLA

In [13]:
r5a = df.groupby(['NO_CINE_AREA_GERAL']).\
agg(QT_CONC=('QT_CONC', 'sum'), QT_ING5=('QT_ING5', 'sum'),
    QT_CONCpv=('QT_CONCpv', 'sum'), QT_ING5pv=('QT_ING5pv', 'sum'),
    QT_CONCpb=('QT_CONCpb', 'sum'), QT_ING5pb=('QT_ING5pb', 'sum')).\
assign(EVASAO = lambda x: dropout(x.QT_CONC, x.QT_ING5),
       EVASAOpv = lambda x: dropout(x.QT_CONCpv, x.QT_ING5pv),
       EVASAOpb = lambda x: dropout(x.QT_CONCpb, x.QT_ING5pb))
r5a.filter(items = ['EVASAO', 'EVASAOpv','EVASAOpb'])

,EVASAO,EVASAOpv,EVASAOpb
NO_CINE_AREA_GERAL,,,
"Ciências naturais, matemática e estatística",61.82,51.4,60.09


# POR REGIÃO

In [14]:
r5c = df.groupby(['NO_REGIAO_IES']).agg(QT_CONC=('QT_CONC', 'sum'), QT_ING5=('QT_ING5', 'sum'),
    QT_CONCpv=('QT_CONCpv', 'sum'), QT_ING5pv=('QT_ING5pv', 'sum'),
    QT_CONCpb=('QT_CONCpb', 'sum'), QT_ING5pb=('QT_ING5pb', 'sum')).\
assign(EVASAO = lambda x: dropout(x.QT_CONC, x.QT_ING5),
       EVASAOpv = lambda x: dropout(x.QT_CONCpv, x.QT_ING5pv),
       EVASAOpb = lambda x: dropout(x.QT_CONCpb, x.QT_ING5pb))
r5c.filter(items = ['EVASAO', 'EVASAOpv','EVASAOpb'])

,EVASAO,EVASAOpv,EVASAOpb
NO_REGIAO_IES,,,
Centro-Oeste,61.67,58.57,61.25
Nordeste,72.24,61.13,71.90
Norte,58.85,58.92,47.14
Sudeste,52.14,40.05,49.95
Sul,66.49,56.42,65.63


# COMPARAÇÃO ENTRE OS ESTADOS

In [15]:
# Ranqueando por evasão de estudantes de escola pública
r6 = df.groupby(['SG_UF_IES']).\
    agg(QT_CONC=('QT_CONC', 'sum'), QT_ING5=('QT_ING5', 'sum'),
    QT_CONCpv=('QT_CONCpv', 'sum'), QT_ING5pv=('QT_ING5pv', 'sum'),
    QT_CONCpb=('QT_CONCpb', 'sum'), QT_ING5pb=('QT_ING5pb', 'sum')).\
assign(EVASAO = lambda x: dropout(x.QT_CONC, x.QT_ING5),
       EVASAOpv = lambda x: dropout(x.QT_CONCpv, x.QT_ING5pv),
       EVASAOpb = lambda x: dropout(x.QT_CONCpb, x.QT_ING5pb))
r6.filter(items = ['EVASAO', 'EVASAOpv','EVASAOpb']).sort_values('EVASAOpb', ascending= False)

,EVASAO,EVASAOpv,EVASAOpb
SG_UF_IES,,,
SE,82.77,79.16,85.61
AL,77.20,66.87,80.10
PB,78.03,69.32,78.89
RO,80.78,37.93,78.57
MT,75.89,74.05,78.23
PI,80.91,52.33,78.19
RR,77.88,0.00,76.12
SC,70.58,69.42,73.87
TO,74.62,85.00,73.02


# ESTATÍSTICA DESCRITIVA DA EVASÃO POR ANO

In [16]:
r7 = df.groupby('ANO').EVASAO.describe().filter(items=['mean', 'std', '50%','min', 'max'])
r7.columns = ['Média', 'Desvio-Padrão', 'Mediana', 'Mínimo', 'Máximo']
r7

,Média,Desvio-Padrão,Mediana,Mínimo,Máximo
ANO,,,,,
2016,60.575385,23.242091,65.040,0.0,97.67
2017,57.320870,23.034575,60.405,0.0,90.58
2018,59.021562,24.254165,63.035,0.0,100.00
2019,56.134571,26.108861,63.180,0.0,97.22
2020,67.106604,24.343398,73.870,0.0,100.00
2021,68.249813,21.130919,72.490,0.0,99.94


# ESTATÍSTICA DESCRITIVA DA EVASÃO POR REGIÃO

In [17]:
r8 = df.groupby('NO_REGIAO_IES').EVASAO.describe().filter(items=['mean', 'std', '50%','min', 'max'])
r8.columns = ['Média', 'Desvio-Padrão', 'Mediana', 'Mínimo', 'Máximo']
r8

,Média,Desvio-Padrão,Mediana,Mínimo,Máximo
NO_REGIAO_IES,,,,,
Centro-Oeste,58.819310,21.832470,61.700,0.0,100.00
Nordeste,71.374940,20.362694,77.120,0.0,100.00
Norte,62.863699,28.892182,70.800,0.0,98.21
Sudeste,54.021146,24.560356,61.125,0.0,97.22
Sul,60.363113,21.201277,64.700,0.0,91.80
